In [ ]:
#andromeda with multi query attention
!git clone https://github.com/kyegomez/Optimus-Prime.git
%cd Optimus-Prime
!pip install --upgrade torch
# !pip install -r requirements.txt
!pip install einops
# !pip install --upgrade torch

# %cd Optimus-Prime
# # %cd examples
# # !ls
# !python3 trainandromeda.py 
# #%cd enwik8_simple
# # !python trainandromeda.py


from torch.serialization import load
import torch 
from x_transformers import TransformerWrapper, Decoder, AutoregressiveWrapper

#training
import random
import tqdm
import gzip
import numpy as np
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
import os
# from torch.utils.tensorboard import SummaryWriter
# from torchmetrics import MetricCollection, Accuracy


# constants

NUM_BATCHES = int(1e5)
BATCH_SIZE = 4
GRADIENT_ACCUMULATE_EVERY = 4
LEARNING_RATE = 1e-4
VALIDATE_EVERY  = 100
GENERATE_EVERY  = 500
GENERATE_LENGTH = 1024
SEQ_LEN = 1024
SAVE_EVERY=500


# helpers

def cycle(loader):
    while True:
        for data in loader:
            yield data

def decode_token(token):
    return str(chr(max(32, token)))

def decode_tokens(tokens):
    return ''.join(list(map(decode_token, tokens)))

model = TransformerWrapper(
    num_tokens=64007,
    max_seq_len=8192,
    use_abs_pos_emb = False,
    attn_layers = Decoder(
        dim=512,
        depth=6,
        heads=8,
        alibi_pos_bias=True,
        alibi_num_heads=4,
        rotary_xpos=True,
        attn_flash = True,
        deepnorm=True,
        shift_tokens=1,
        attn_one_kv_head = True,
        #qk_norm=True
    )
)


model = AutoregressiveWrapper(model)
model.cuda()

with gzip.open('./enwik8.gz') as file:
  data = np.frombuffer(file.read(int(95e6)), dtype=np.uint8).copy()
  train_x, valid_x = np.split(data, [int(90e6)])
  data_train, data_val = torch.from_numpy(train_x), torch.from_numpy(valid_x) #.cuda()??

class TextSamplerDataset(Dataset):
    def __init__(self, data, seq_len):
        super().__init__()
        self.data = data
        self.seq_len = seq_len

    def __getitem__(self, index):
        rand_start = torch.randint(0, self.data.size(0) - self.seq_len - 1, (1,))
        full_seq = self.data[rand_start: rand_start + self.seq_len + 1].long()
        return full_seq.cuda()

    def __len__(self):
        return self.data.size(0) // self.seq_len

train_dataset = TextSamplerDataset(data_train, SEQ_LEN)
val_dataset   = TextSamplerDataset(data_val, SEQ_LEN)
train_loader  = cycle(DataLoader(train_dataset, batch_size = BATCH_SIZE, drop_last = True))
val_loader    = cycle(DataLoader(val_dataset, batch_size = BATCH_SIZE, drop_last = True))

# optimizer

optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# training

# #init tensorboard 
# writer = SummaryWriter(log_dir="./log")

# #define metrics
# metrics = MetricCollection({'accuracy': Accuracy(num_classes=num_classes, task='classification')})
device="cuda"
for i in tqdm.tqdm(range(NUM_BATCHES), mininterval=10., desc='training'):
    model.train()

    for _ in range(GRADIENT_ACCUMULATE_EVERY):
        loss = model(next(train_loader))#.to(device)
        (loss / GRADIENT_ACCUMULATE_EVERY).backward()#.to(device)#.cuda()

    print(f'training loss: {loss.item()}')
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optim.step()
    optim.zero_grad()


    if i % VALIDATE_EVERY == 0:
            model.eval()
            with torch.no_grad():
                loss = model(next(val_loader))
                print(f'validation loss: {loss.item()}')

                # # Calculate validation metrics
                # val_metrics = MetricCollection({'val_accuracy': Accuracy()})
                # val_metrics(loss, model(next(val_loader)).argmax(dim=-1))

                # # Add validation metrics to the SummaryWriter
                # writer.add_scalar('Validation/Accuracy', val_metrics['val_accuracy'].compute(), global_step=i)

    if i % GENERATE_EVERY == 0:
        model.eval()
        inp = random.choice(val_dataset)[:-1]
        prime = decode_tokens(inp)
        print(f'%s \n\n %s', (prime, '*' * 100))

        sample = model.generate(inp, GENERATE_LENGTH)
        output_str = decode_tokens(sample)
        print(output_str)

    # Save the model every save_every iterations
    if i % SAVE_EVERY == 0:
        # Specify the directory and filename to save the model
        save_dir = './saved_models/'
        save_filename = 'model_checkpoint.pt'

        # Create the save directory if it doesn't exist
        os.makedirs(save_dir, exist_ok=True)

        # Save the model checkpoint
        torch.save(model.state_dict(), os.path.join(save_dir, save_filename))
        print(f"Model saved at iteration {i}")

#     # Add training metrics to the SummaryWriter
#     writer.add_scalar('Training/Accuracy', metrics['accuracy'].compute(), global_step=i)

#     # Close the SummaryWriter
# writer.close()

Cloning into 'Optimus-Prime'...
remote: Enumerating objects: 1642, done.
remote: Counting objects: 100% (481/481), done.
remote: Compressing objects: 100% (210/210), done.
remote: Total 1642 (delta 324), reused 388 (delta 268), pack-reused 1161
Receiving objects: 100% (1642/1642), 37.52 MiB | 23.85 MiB/s, done.
Resolving deltas: 100% (1115/1115), done.
/content/Optimus-Prime
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 98.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

training:   0%|          | 0/100000 [00:00<?, ?it/s]

training loss: 11.18986701965332
validation loss: 11.040206909179688
%s 

 %s ("lato]].  The Hesychast interprets Christ's injunction in the [[Gospel of Matthew]] to &quot;go into your closet to pray&quot;, to mean that he should ignore the senses and withdraw inward.  St John of Sinai writes:  &quot;Hesychasm is the enclosing of the bodiless mind ''(nous)'' in the bodily house of the body.&quot;  ''(Ladder,'' Step 27, 5, (Step 27, 6 in the Holy Transfiguration edition).)  In Step 27, 21 of the ''Ladder'' (Step 27, 22â\x80\x933 of the Holy Transfiguration edition), St John of Sinai describes Hesychast practice as follows:  ::Take up your seat on a high place and watch, if only you know how, and then you will see in what manner, when, whence, how many and what kind of thieves come to enter and steal your clusters of grapes.  When the watchman grows weary, he stands up and prays; and then he sits down again and courageously takes up his former task.  In this passage, St John of Sinai say

training:   0%|          | 1/100000 [01:19<2215:26:50, 79.76s/it]ERROR:tornado.application:Exception in callback functools.partial(<bound method OutStream._flush of <ipykernel.iostream.OutStream object at 0x7fbfb3826830>>)
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 738, in _run_callback
    ret = callback()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/iostream.py", line 381, in _flush
    self.session.send(self.pub_thread, 'stream', content=content,
  File "/usr/local/lib/python3.10/dist-packages/jupyter_client/session.py", line 742, in send
    to_send = self.serialize(msg, ident)
  File "/usr/local/lib/python3.10/dist-packages/jupyter_client/session.py", line 630, in serialize
    content = self.pack(content)
  File "/usr/local/lib/python3.10/dist-packages/jupyter_client/session.py", line 82, in <lambda>
    json_packer = lambda obj: jsonapi.dumps(obj, default=date_default,
  File "/usr/local/lib/python3.10

Model saved at iteration 0


...

training loss: 0.8769919872283936
training loss: 1.044207215309143
training loss: 1.1802260875701904
training loss: 1.2171916961669922


training:   9%|▉         | 9281/100000 [8:27:39<78:34:37,  3.12s/it]

training loss: 1.0807397365570068
training loss: 1.1084022521972656
training loss: 1.1412968635559082
training loss: 1.0119473934173584


training:   9%|▉         | 9285/100000 [8:27:51<78:34:00,  3.12s/it]

training loss: 1.0761487483978271


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [3]:
# Download the results
zip_filename = 'model_checkpoint.pt'
if os.path.exists(zip_filename):
  os.remove(zip_filename)
os.system(f"zip -r -j {zip_filename} results/*")
files.download(zip_filename)

/bin/bash: nvidia-smi: command not found
